Libraries

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

Data Path

In [44]:
data_dir = "/content/drive/MyDrive/Final data set ggg"
img_height, img_width = 150, 150  # You can adjust these dimensions as needed.
batch_size = 32

Data augmentation

In [45]:
# Create data generators with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 913 images belonging to 3 classes.
Found 227 images belonging to 3 classes.


**Model( Using DenseNet201 as base model)**

In [46]:
# Create a pre-trained DenseNet201 model to extract features
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers:
    layer.trainable = False

In [47]:
# Extract features for the training data
train_features = base_model.predict(train_generator)
train_labels = train_generator.classes


29/29 [==============================] - 74s 3s/step


In [48]:
# Extract features for the validation data
validation_features = base_model.predict(validation_generator)
validation_labels = validation_generator.classes


8/8 [==============================] - 32s 5s/step


In [49]:
# Reshape the features for XGBoost (2D matrix)
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)


Train XGBoost

In [50]:
# Train an XGBoost classifier on the reshaped features
xgb_model = XGBClassifier()
xgb_model.fit(train_features, train_labels)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Validation

In [51]:
# Make predictions on the validation set
validation_predictions = xgb_model.predict(validation_features)
validation_accuracy = accuracy_score(validation_labels, validation_predictions)
print("Validation accuracy:", validation_accuracy)


Validation accuracy: 0.933920704845815


Save Model

In [52]:
# Save the XGBoost model
xgb_model.save_model("/content/drive/MyDrive/GLUCOMA/models/xgboost_model_100x100.model")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:20:03] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


Make Prediction on new Single image

In [53]:
from tensorflow.keras.applications import DenseNet201
import numpy as np
from PIL import Image
from xgboost import XGBClassifier

# Load your XGBoost model (already trained)
xgb_model = XGBClassifier()
xgb_model.load_model("/content/drive/MyDrive/GLUCOMA/models/xgboost_model_150x150.model")  # Load your trained model file

# Load the new image
new_image_path = "/content/drive/MyDrive/GLUCOMA/test/Negative/glaucoma 150_255.jpg"
img = Image.open(new_image_path)
img = img.resize((150, 150))  # Resize to match the input size of the model

# Create a pre-trained DenseNet201 model to extract features
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Extract features from the new image
img_array = np.array(img)
img_array = img_array / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
features = base_model.predict(img_array)

# Reshape the features for XGBoost (2D matrix)
features = features.reshape(features.shape[0], -1)

# Make predictions using the XGBoost model
predictions = xgb_model.predict(features)

# Assuming 1 indicates Glaucoma and 0 indicates Non-Glaucoma
if predictions[0] == 1:
    print("The image is predicted to have Glaucoma.")
else:
    print("The image is predicted to be Non-Glaucoma.")


1/1 [==============================] - 3s 3s/step
The image is predicted to be Non-Glaucoma.


Make Predictionn on multiple images

In [41]:
import os
import numpy as np
from PIL import Image
from xgboost import XGBClassifier
from tensorflow.keras.applications import DenseNet201

# Load your trained XGBoost model
xgb_model = XGBClassifier()
xgb_model.load_model("/content/drive/MyDrive/GLUCOMA/models/xgboost_model_150x150.model")  # Load your trained model file

# Define the directory containing the test images
test_dir = "/content/drive/MyDrive/GLUCOMA/test"

# Iterate through the images and make predictions
for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)

        # Load and preprocess the image
        img = Image.open(image_path)
        img = img.resize((150, 150))
        img_array = np.array(img)
        img_array = img_array / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Extract features from the image using DenseNet201
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
        features = base_model.predict(img_array)
        features = features.reshape(features.shape[0], -1)

        # Make predictions using the XGBoost model
        predictions = xgb_model.predict(features)

        # Define class labels
        if predictions[0] == 1:
            predicted_label = "Glaucoma"
        else:
            predicted_label = "Non-Glaucoma"

        # Display the predicted label and actual label
        print(f"Predicted: {predicted_label}, Actual: {class_name}")



1/1 [==============================] - 3s 3s/step
Predicted: Non-Glaucoma, Actual: Positive
1/1 [==============================] - 4s 4s/step
Predicted: Glaucoma, Actual: Positive
1/1 [==============================] - 5s 5s/step
Predicted: Glaucoma, Actual: Positive
1/1 [==============================] - 4s 4s/step
Predicted: Non-Glaucoma, Actual: Positive
1/1 [==============================] - 4s 4s/step
Predicted: Glaucoma, Actual: Positive
1/1 [==============================] - 4s 4s/step
Predicted: Glaucoma, Actual: Positive
1/1 [==============================] - 3s 3s/step
Predicted: Non-Glaucoma, Actual: Positive
1/1 [==============================] - 4s 4s/step
Predicted: Glaucoma, Actual: Positive
1/1 [==============================] - 3s 3s/step
Predicted: Glaucoma, Actual: Positive
1/1 [==============================] - 3s 3s/step
Predicted: Non-Glaucoma, Actual: Positive
1/1 [==============================] - 4s 4s/step
Predicted: Glaucoma, Actual: Positive
1/1 [===========